In [ ]:
import orsopy
import matplotlib.pyplot as plt

import scipp as sc
import plopp as pp
import sciline
from ess import amor
from ess.reflectometry.types import *
from ess.amor.types import ChopperPhase

## Recreate reference files using Jochens Amor data reduction software

#### step 1
```
%%bash
git clone https://github.com/jochenstahn/amor.git
wget -P raw https://public.esss.dk/groups/scipp/ess/amor/1/amor2023n000{608..614}.hdf
```

#### step 2
```
%%bash
args='-Y 2023 -d raw -n 614 -m 0.05 -co -7.5 -of ort'

rm raw/614.norm
python amor/neos.py $args -f 614

for i in `seq 608 613`
do
    python amor/neos.py $args -f $i -o $i $args 
done
```

## Run essreflectometry on the same files

In [ ]:
pl = sciline.Pipeline(
    (*amor.providers, *amor.data.providers),
    params=amor.default_parameters,
)

pl[SampleSize[Sample]] = sc.scalar(10.0, unit='mm')
pl[SampleSize[Reference]] = sc.scalar(10.0, unit='mm')
pl[ChopperPhase[Sample]] = sc.scalar(-7.5, unit='deg')

# In Jochens workflow:
#    * divergence mask: [-0.7, 0.7]
#    * note, divergence relative to beam center
pl[WavelengthBins] = sc.geomspace('wavelength', 2.8, 12.5, 300, unit='angstrom')
pl[QBins] = sc.geomspace(
    dim='Q', start=0.00505, stop=2.93164766e-01, num=391, unit='1/angstrom'
)
pl[YIndexLimits] = sc.scalar(11, unit=None), sc.scalar(41, unit=None)
pl[ZIndexLimits] = sc.scalar(80, unit=None), sc.scalar(370, unit=None)

# Chopper phase value in the file is wrong, so we set it manually
pl[ChopperPhase[Reference]] = sc.scalar(-7.5, unit='deg')
# The sample rotation value in the file is slightly off, so we set it manually
pl[SampleRotation[Reference]] = sc.scalar(0.65, unit='deg')
pl[TutorialFilename[Reference]] = "amor2023n000614.hdf"

reference_result = pl.compute(IdealReferenceIntensity)
# Set the result back onto the pipeline to cache it
pl[IdealReferenceIntensity] = reference_result

In [ ]:
mu = {
    '608': 0.8,
    '609': 2.2,
    '610': 3.6,
    '611': 5.0,
    '612': 0.6,
    '613': 0.6,
}

results = sc.DataGroup({'ess': sc.DataGroup(), 'psi': sc.DataGroup()})

# ESS results
for key, angle in mu.items():
    print(key, '... ', end='')
    pl[SampleRotation[Sample]] = sc.scalar(angle + 0.05, unit='deg')
    pl[TutorialFilename[Sample]] = f"amor2023n000{key}.hdf"
    da = pl.compute(NormalizedIofQ).hist()
    da.coords['Q'] = sc.midpoints(da.coords['Q'], dim='Q')
    results['ess'][key] = da
    print('Done!')

In [ ]:
# PSI results
for key, angle in mu.items():
    d = orsopy.fileio.load_orso(amor.data.get_path(f'{key}.Rqz.ort'))[0].data
    y = sc.array(dims=['Q'], values=d[:, 1], variances=d[:, 2] ** 2)
    results['psi'][key] = sc.DataArray(
        data=y, coords={'Q': sc.array(dims=['Q'], values=d[:, 0], unit='1/angstrom')}
    )

## Plot reflectivity curve comparison

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 12))

for ax, (key, angle) in zip(axs.ravel(), mu.items()):
    pp.plot(
        {
            facility: results[facility][key][results[facility][key].data > 0.0]
            for facility in results
        },
        ax=ax,
        norm='log',
        ls='solid',
        marker=None,
        vmin=1.0e-4,
        title=f'Angle: {angle} deg',
    )

fig.tight_layout()

## Overlay curves from same sample at different angle, should be on top of each other

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

for ax, (facility, res) in zip(axs.ravel(), results.items()):
    pp.plot(
        {k: v for k, v in list(res.items())[:4]},  # First 4 curves
        ax=ax,
        norm='log',
        ls='solid',
        marker=None,
        vmin=1.0e-4,
        title=facility,
    )